In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import os

from explore import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler, MinMaxScaler
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_regression, RFE

import warnings
warnings.filterwarnings("ignore")

### Acquire Teams

In [2]:
teams = pd.read_csv('teams.csv')
teams

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario
8,0,1610612747,1948,2019,LAL,Lakers,1948,Los Angeles,Staples Center,19060.0,Jerry Buss Family Trust,Rob Pelinka,Frank Vogel,South Bay Lakers
9,0,1610612748,1988,2019,MIA,Heat,1988,Miami,AmericanAirlines Arena,19600.0,Micky Arison,Pat Riley,Erik Spoelstra,Sioux Falls Skyforce


### Acquire Stats for Each Game

In [3]:
games = pd.read_csv('games.csv')
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,1610612755,1610612764,2020,1610612755,120.0,0.557,0.684,...,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,1610612752,1610612737,2020,1610612752,101.0,0.383,0.739,...,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,1610612762,1610612763,2020,1610612762,141.0,0.544,0.774,...,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1
3,2021-05-25,42000112,Final,1610612751,1610612738,2020,1610612751,130.0,0.523,0.955,...,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1
4,2021-05-25,42000152,Final,1610612756,1610612747,2020,1610612756,102.0,0.465,0.933,...,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0


### Cleaning Data

In [4]:
# Dropping HOME_TEAM_ID and VISITOR_TEAM_ID since their numbers are the same as
# TEAM_ID_home and TEAM_ID_away
games.drop(columns = ['HOME_TEAM_ID','VISITOR_TEAM_ID'], inplace=True)

In [5]:
# Changing Game_Date_EST to datetime
games['GAME_DATE_EST']= pd.to_datetime(games['GAME_DATE_EST'])
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24677 entries, 0 to 24676
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_DATE_EST     24677 non-null  datetime64[ns]
 1   GAME_ID           24677 non-null  int64         
 2   GAME_STATUS_TEXT  24677 non-null  object        
 3   SEASON            24677 non-null  int64         
 4   TEAM_ID_home      24677 non-null  int64         
 5   PTS_home          24578 non-null  float64       
 6   FG_PCT_home       24578 non-null  float64       
 7   FT_PCT_home       24578 non-null  float64       
 8   FG3_PCT_home      24578 non-null  float64       
 9   AST_home          24578 non-null  float64       
 10  REB_home          24578 non-null  float64       
 11  TEAM_ID_away      24677 non-null  int64         
 12  PTS_away          24578 non-null  float64       
 13  FG_PCT_away       24578 non-null  float64       
 14  FT_PCT_away       2457

In [6]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,2020,1610612755,120.0,0.557,0.684,0.429,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,2020,1610612752,101.0,0.383,0.739,0.364,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,2020,1610612762,141.0,0.544,0.774,0.487,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1
3,2021-05-25,42000112,Final,2020,1610612751,130.0,0.523,0.955,0.447,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1
4,2021-05-25,42000152,Final,2020,1610612756,102.0,0.465,0.933,0.308,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0


In [7]:
# We can drop this column
games.GAME_STATUS_TEXT.value_counts()

Final    24677
Name: GAME_STATUS_TEXT, dtype: int64

In [8]:
games.drop(columns = 'GAME_STATUS_TEXT', inplace=True)
games.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home',
       'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'TEAM_ID_away', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away',
       'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [9]:
# checking for nulls
games.isnull().any()

GAME_DATE_EST     False
GAME_ID           False
SEASON            False
TEAM_ID_home      False
PTS_home           True
FG_PCT_home        True
FT_PCT_home        True
FG3_PCT_home       True
AST_home           True
REB_home           True
TEAM_ID_away      False
PTS_away           True
FG_PCT_away        True
FT_PCT_away        True
FG3_PCT_away       True
AST_away           True
REB_away           True
HOME_TEAM_WINS    False
dtype: bool

In [10]:
# This tells us the null values for these games are in October 
# There are only 99 rows out of 24,677...I think it's okay to drop

games[games.PTS_home.isnull()==True].GAME_DATE_EST.value_counts()

2003-10-14    10
2003-10-17     9
2003-10-11     9
2003-10-24     9
2003-10-22     8
2003-10-18     7
2003-10-16     6
2003-10-23     6
2003-10-21     5
2003-10-19     5
2003-10-10     5
2003-10-15     4
2003-10-20     4
2003-10-09     4
2003-10-12     3
2003-10-08     2
2003-10-13     2
2003-10-07     1
Name: GAME_DATE_EST, dtype: int64

In [11]:
games.drop(games[games.PTS_home.isnull()==True].index, inplace = True)
games.shape

(24578, 18)

In [12]:
# Now we don't have any nulls!
games.isnull().any()

GAME_DATE_EST     False
GAME_ID           False
SEASON            False
TEAM_ID_home      False
PTS_home          False
FG_PCT_home       False
FT_PCT_home       False
FG3_PCT_home      False
AST_home          False
REB_home          False
TEAM_ID_away      False
PTS_away          False
FG_PCT_away       False
FT_PCT_away       False
FG3_PCT_away      False
AST_away          False
REB_away          False
HOME_TEAM_WINS    False
dtype: bool

In [13]:
# Creating the total points column, this will be our target
games['total_points'] = games.PTS_home + games.PTS_away

In [14]:
games.head()

,GAME_DATE_EST,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points
0,2021-05-26,42000102,2020,1610612755,120.0,0.557,0.684,0.429,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1,215.0
1,2021-05-26,42000132,2020,1610612752,101.0,0.383,0.739,0.364,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1,193.0
2,2021-05-26,42000142,2020,1610612762,141.0,0.544,0.774,0.487,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1,270.0
3,2021-05-25,42000112,2020,1610612751,130.0,0.523,0.955,0.447,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1,238.0
4,2021-05-25,42000152,2020,1610612756,102.0,0.465,0.933,0.308,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0,211.0


In [15]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24578 entries, 0 to 24676
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   GAME_DATE_EST   24578 non-null  datetime64[ns]
 1   GAME_ID         24578 non-null  int64         
 2   SEASON          24578 non-null  int64         
 3   TEAM_ID_home    24578 non-null  int64         
 4   PTS_home        24578 non-null  float64       
 5   FG_PCT_home     24578 non-null  float64       
 6   FT_PCT_home     24578 non-null  float64       
 7   FG3_PCT_home    24578 non-null  float64       
 8   AST_home        24578 non-null  float64       
 9   REB_home        24578 non-null  float64       
 10  TEAM_ID_away    24578 non-null  int64         
 11  PTS_away        24578 non-null  float64       
 12  FG_PCT_away     24578 non-null  float64       
 13  FT_PCT_away     24578 non-null  float64       
 14  FG3_PCT_away    24578 non-null  float64       
 15  AS

In [16]:
df = games.set_index('GAME_DATE_EST').sort_index()

In [17]:
df

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points
GAME_DATE_EST,,,,,,,,,,,,,,,,,,
2003-10-05,10300001,2003,1610612762,90.0,0.457,0.735,0.143,23.0,41.0,1610612742,85.0,0.447,0.500,0.250,20.0,38.0,1,175.0
2003-10-06,10300002,2003,1610612763,105.0,0.494,0.618,0.267,25.0,48.0,1610612749,94.0,0.427,0.700,0.154,20.0,43.0,1,199.0
2003-10-07,10300009,2003,1610612758,101.0,0.467,0.871,0.444,19.0,39.0,1610612746,82.0,0.368,0.609,0.364,13.0,50.0,1,183.0
2003-10-07,10300005,2003,1610612757,104.0,0.527,0.657,0.429,22.0,33.0,1610612745,80.0,0.470,0.667,0.333,10.0,37.0,1,184.0
2003-10-07,10300007,2003,1610612748,86.0,0.352,0.647,0.222,15.0,55.0,1610612755,79.0,0.329,0.897,0.143,7.0,44.0,1,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25,42000152,2020,1610612756,102.0,0.465,0.933,0.308,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0,211.0
2021-05-25,42000112,2020,1610612751,130.0,0.523,0.955,0.447,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1,238.0
2021-05-26,42000142,2020,1610612762,141.0,0.544,0.774,0.487,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1,270.0


### Creating readable team labels

In [18]:
def label_teams_away(df):
    if df['TEAM_ID_away'] == 1610612737:
        return 'ATL'
    if df['TEAM_ID_away'] == 1610612738:
        return 'BOS'
    if df['TEAM_ID_away'] == 1610612739:
        return 'CLE'
    if df['TEAM_ID_away'] == 1610612740:
        return 'NOP'
    if df['TEAM_ID_away']  == 1610612741:
        return 'CHI'
    if df['TEAM_ID_away'] == 1610612742:
        return 'DAL'
    if df['TEAM_ID_away'] == 1610612743:
        return 'DEN'
    if df['TEAM_ID_away'] == 1610612744:
        return 'GSW'
    if df['TEAM_ID_away'] == 1610612745:
        return 'HOU'
    if df['TEAM_ID_away'] == 1610612746:
        return 'LAC'
    if df['TEAM_ID_away'] == 1610612747:
        return 'LAL'
    if df['TEAM_ID_away'] == 1610612748:
        return 'MIA'
    if df['TEAM_ID_away'] == 1610612749:
        return 'MIL'
    if df['TEAM_ID_away'] == 1610612750:
        return 'MIN'
    if df['TEAM_ID_away'] == 1610612751:
        return 'BKN'
    if df['TEAM_ID_away'] == 1610612752:
        return 'NYK'
    if df['TEAM_ID_away'] == 1610612753:
        return 'ORL'
    if df['TEAM_ID_away'] == 1610612754:
        return 'IND'
    if df['TEAM_ID_away'] == 1610612755:
        return 'PHI'
    if df['TEAM_ID_away'] == 1610612756:
        return 'PHX'
    if df['TEAM_ID_away'] == 1610612757:
        return 'POR'
    if df['TEAM_ID_away'] == 1610612758:
        return 'SAC'
    if df['TEAM_ID_away'] == 1610612759:
        return 'SAS'
    if df['TEAM_ID_away'] == 1610612760:
        return 'OKC'
    if df['TEAM_ID_away'] == 1610612761:
        return 'TOR'
    if df['TEAM_ID_away'] == 1610612762:
        return 'UTA'
    if df['TEAM_ID_away'] == 1610612763:
        return 'MEM'
    if df['TEAM_ID_away'] == 1610612764:
        return 'WAS'
    if df['TEAM_ID_away'] == 1610612765:
        return 'DET'
    if df['TEAM_ID_away'] == 1610612766:
        return 'CHA'
    return 'Other'

In [19]:
def label_teams_home(df):
    if df['TEAM_ID_home'] == 1610612737:
        return 'ATL'
    if df['TEAM_ID_home'] == 1610612738:
        return 'BOS'
    if df['TEAM_ID_home'] == 1610612739:
        return 'CLE'
    if df['TEAM_ID_home'] == 1610612740:
        return 'NOP'
    if df['TEAM_ID_home']  == 1610612741:
        return 'CHI'
    if df['TEAM_ID_home'] == 1610612742:
        return 'DAL'
    if df['TEAM_ID_home'] == 1610612743:
        return 'DEN'
    if df['TEAM_ID_home'] == 1610612744:
        return 'GSW'
    if df['TEAM_ID_home'] == 1610612745:
        return 'HOU'
    if df['TEAM_ID_home'] == 1610612746:
        return 'LAC'
    if df['TEAM_ID_home'] == 1610612747:
        return 'LAL'
    if df['TEAM_ID_home'] == 1610612748:
        return 'MIA'
    if df['TEAM_ID_home'] == 1610612749:
        return 'MIL'
    if df['TEAM_ID_home'] == 1610612750:
        return 'MIN'
    if df['TEAM_ID_home'] == 1610612751:
        return 'BKN'
    if df['TEAM_ID_home'] == 1610612752:
        return 'NYK'
    if df['TEAM_ID_home'] == 1610612753:
        return 'ORL'
    if df['TEAM_ID_home'] == 1610612754:
        return 'IND'
    if df['TEAM_ID_home'] == 1610612755:
        return 'PHI'
    if df['TEAM_ID_home'] == 1610612756:
        return 'PHX'
    if df['TEAM_ID_home'] == 1610612757:
        return 'POR'
    if df['TEAM_ID_home'] == 1610612758:
        return 'SAC'
    if df['TEAM_ID_home'] == 1610612759:
        return 'SAS'
    if df['TEAM_ID_home'] == 1610612760:
        return 'OKC'
    if df['TEAM_ID_home'] == 1610612761:
        return 'TOR'
    if df['TEAM_ID_home'] == 1610612762:
        return 'UTA'
    if df['TEAM_ID_home'] == 1610612763:
        return 'MEM'
    if df['TEAM_ID_home'] == 1610612764:
        return 'WAS'
    if df['TEAM_ID_home'] == 1610612765:
        return 'DET'
    if df['TEAM_ID_home'] == 1610612766:
        return 'CHA'
    return 'Other'

In [20]:
df['home_team'] = df.apply(lambda df: label_teams_home(df), axis=1)

In [21]:
df['away_team'] = df.apply(lambda df: label_teams_away(df), axis=1)

In [22]:
df.head()

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points,home_team,away_team
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,
2003-10-05,10300001,2003,1610612762,90.0,0.457,0.735,0.143,23.0,41.0,1610612742,85.0,0.447,0.500,0.250,20.0,38.0,1,175.0,UTA,DAL
2003-10-06,10300002,2003,1610612763,105.0,0.494,0.618,0.267,25.0,48.0,1610612749,94.0,0.427,0.700,0.154,20.0,43.0,1,199.0,MEM,MIL
2003-10-07,10300009,2003,1610612758,101.0,0.467,0.871,0.444,19.0,39.0,1610612746,82.0,0.368,0.609,0.364,13.0,50.0,1,183.0,SAC,LAC
2003-10-07,10300005,2003,1610612757,104.0,0.527,0.657,0.429,22.0,33.0,1610612745,80.0,0.470,0.667,0.333,10.0,37.0,1,184.0,POR,HOU
2003-10-07,10300007,2003,1610612748,86.0,0.352,0.647,0.222,15.0,55.0,1610612755,79.0,0.329,0.897,0.143,7.0,44.0,1,165.0,MIA,PHI


## Engineer Average Features
I won't be able to use these actual stats for my model, but I can create rolling averages 

#### Create the DF that I will use for Train, Validate, Test

- I will use the 2018-2019 for my test since it was the last full season. 2019-2020 was suspended due to covid, and 2020-2021 was shortened. 

- I'll validate on 2017-2018 and train on 3 seasons prior

- I'll add one extra season (2013-2014) so I can get the averages going before my train starts

- In total: 2013-2019 (Seasons 2013, 2014, 2015, 2016, 2017, 2018)

In [23]:
df = df[(df.SEASON == 2013)|(df.SEASON == 2014)|(df.SEASON == 2015)|(df.SEASON == 2016)|(df.SEASON == 2017)|(df.SEASON == 2018)]

In [24]:
spurs = df[(df.home_team == 'SAS')|(df.away_team == 'SAS')]

#### Created a spurs score for each game

In [25]:
spurs['spurs_score'] = spurs.apply(lambda x: x.PTS_home if x.home_team == 'SAS' else x.PTS_away, axis=1)

In [26]:
spurs

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points,home_team,away_team,spurs_score
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,0.461,0.667,0.370,22.0,45.0,0,205.0,SAS,PHX,99.0
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,0.466,0.588,0.500,24.0,50.0,1,192.0,DEN,SAS,94.0
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,0.477,0.813,0.346,23.0,47.0,0,210.0,ATL,SAS,106.0
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,0.411,0.926,0.458,24.0,40.0,1,217.0,MIA,SAS,96.0
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,0.447,0.889,0.529,22.0,42.0,1,224.0,SAS,ORL,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-18,41800153,2018,1610612759,118.0,0.494,0.719,0.389,24.0,45.0,1610612743,...,0.519,0.529,0.517,31.0,37.0,1,226.0,SAS,DEN,118.0
2019-04-20,41800154,2018,1610612759,103.0,0.468,0.750,0.294,23.0,46.0,1610612743,...,0.449,0.917,0.484,25.0,45.0,0,220.0,SAS,DEN,103.0
2019-04-23,41800155,2018,1610612743,108.0,0.500,0.800,0.424,25.0,49.0,1610612759,...,0.411,0.600,0.292,17.0,42.0,1,198.0,DEN,SAS,90.0


#### Create a rolling Average Points of the past 5 games

In [27]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_pts_home'] = spurs_home['PTS_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_pts_away'] = spurs_away['PTS_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points,home_team,away_team,spurs_score,5_game_avg_pts_home,5_game_avg_pts_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,0.370,22.0,45.0,0,205.0,SAS,PHX,99.0,0.0,0.0
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,0.500,24.0,50.0,1,192.0,DEN,SAS,94.0,0.0,0.0
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,0.346,23.0,47.0,0,210.0,ATL,SAS,106.0,0.0,0.0
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,0.458,24.0,40.0,1,217.0,MIA,SAS,96.0,0.0,0.0
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,0.529,22.0,42.0,1,224.0,SAS,ORL,123.0,0.0,0.0
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,0.342,27.0,51.0,0,201.0,SAS,HOU,92.0,0.0,0.0
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,0.533,24.0,44.0,1,195.0,SAS,MEM,101.0,0.0,0.0
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,0.238,20.0,50.0,0,176.0,LAL,SAS,91.0,0.0,0.0
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,0.500,24.0,35.0,1,220.0,POR,SAS,105.0,0.0,98.4


#### Create rolling Average Field Goal Percentage

In [28]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_fg_pct_home'] = spurs_home['FG_PCT_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_fg_pct_away'] = spurs_away['FG_PCT_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,REB_away,HOME_TEAM_WINS,total_points,home_team,away_team,spurs_score,5_game_avg_pts_home,5_game_avg_pts_away,5_game_avg_fg_pct_home,5_game_avg_fg_pct_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,45.0,0,205.0,SAS,PHX,99.0,0.0,0.0,0.0000,0.0000
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,50.0,1,192.0,DEN,SAS,94.0,0.0,0.0,0.0000,0.0000
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,47.0,0,210.0,ATL,SAS,106.0,0.0,0.0,0.0000,0.0000
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,40.0,1,217.0,MIA,SAS,96.0,0.0,0.0,0.0000,0.0000
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,42.0,1,224.0,SAS,ORL,123.0,0.0,0.0,0.0000,0.0000
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,51.0,0,201.0,SAS,HOU,92.0,0.0,0.0,0.0000,0.0000
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,44.0,1,195.0,SAS,MEM,101.0,0.0,0.0,0.0000,0.0000
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,50.0,0,176.0,LAL,SAS,91.0,0.0,0.0,0.0000,0.0000
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,35.0,1,220.0,POR,SAS,105.0,0.0,98.4,0.0000,0.4516


#### Create rolling Average Free Throw Percentage

In [29]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_ft_pct_home'] = spurs_home['FT_PCT_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_ft_pct_away'] = spurs_away['FT_PCT_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,total_points,home_team,away_team,spurs_score,5_game_avg_pts_home,5_game_avg_pts_away,5_game_avg_fg_pct_home,5_game_avg_fg_pct_away,5_game_avg_ft_pct_home,5_game_avg_ft_pct_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,205.0,SAS,PHX,99.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,192.0,DEN,SAS,94.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,210.0,ATL,SAS,106.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,217.0,MIA,SAS,96.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,224.0,SAS,ORL,123.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,201.0,SAS,HOU,92.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,195.0,SAS,MEM,101.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,176.0,LAL,SAS,91.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,220.0,POR,SAS,105.0,0.0,98.4,0.0000,0.4516,0.0000,0.7332


#### Create rolling Average 3 Point Field Goal Percentage

In [30]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_fg3_pct_home'] = spurs_home['FG3_PCT_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_fg3_pct_away'] = spurs_away['FG3_PCT_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,away_team,spurs_score,5_game_avg_pts_home,5_game_avg_pts_away,5_game_avg_fg_pct_home,5_game_avg_fg_pct_away,5_game_avg_ft_pct_home,5_game_avg_ft_pct_away,5_game_avg_fg3_pct_home,5_game_avg_fg3_pct_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,PHX,99.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,SAS,94.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,SAS,106.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,SAS,96.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,ORL,123.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,HOU,92.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,MEM,101.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,SAS,91.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,SAS,105.0,0.0,98.4,0.0000,0.4516,0.0000,0.7332,0.0000,0.4084


#### Create rolling Average Assists

In [38]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_assists_home'] = spurs_home['AST_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_assists_away'] = spurs_away['AST_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,5_game_avg_pts_home,5_game_avg_pts_away,5_game_avg_fg_pct_home,5_game_avg_fg_pct_away,5_game_avg_ft_pct_home,5_game_avg_ft_pct_away,5_game_avg_fg3_pct_home,5_game_avg_fg3_pct_away,5_game_avg_assists_home,5_game_avg_assists_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,0.0,98.4,0.0000,0.4516,0.0000,0.7332,0.0000,0.4084,0.0,23.0


#### Create rolling Average Rebounds

In [39]:
# Create home DF
spurs_home = spurs[spurs.home_team=='SAS']

# Create a rolling average of the past 5 games at home
spurs_home['5_game_avg_rebounds_home'] = spurs_home['REB_home'].rolling(5).mean()

# Create away DF
spurs_away = spurs[spurs.away_team == 'SAS']

# Create a rolling average of the past 5 games away
spurs_away['5_game_avg_rebounds_away'] = spurs_away['REB_away'].rolling(5).mean()

# Concat the two df's together
spurs_avg_pts = [spurs_home,spurs_away]
spurs = pd.concat(spurs_avg_pts)

# Sort the index
spurs.sort_index(inplace=True)

# Forward fill NaN's with the averages 
spurs.fillna(method='ffill',inplace=True)

# Fill rest of the Nan's with zeros
spurs.fillna(0, inplace=True)

spurs.head(15)

,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,...,5_game_avg_fg_pct_home,5_game_avg_fg_pct_away,5_game_avg_ft_pct_home,5_game_avg_ft_pct_away,5_game_avg_fg3_pct_home,5_game_avg_fg3_pct_away,5_game_avg_assists_home,5_game_avg_assists_away,5_game_avg_rebounds_home,5_game_avg_rebounds_away
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,,
2013-10-13,11300045,2013,1610612759,99.0,0.451,0.826,0.333,27.0,39.0,1610612756,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-14,11300048,2013,1610612743,98.0,0.409,0.630,0.208,22.0,51.0,1610612759,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-17,11300068,2013,1610612737,104.0,0.453,0.655,0.333,30.0,42.0,1610612759,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-19,11300076,2013,1610612748,121.0,0.568,0.727,0.464,25.0,39.0,1610612759,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-22,11300091,2013,1610612759,123.0,0.534,0.818,0.423,28.0,45.0,1610612753,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-24,11300105,2013,1610612759,92.0,0.398,0.882,0.389,26.0,41.0,1610612745,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-10-30,21300013,2013,1610612759,101.0,0.526,0.625,0.550,25.0,40.0,1610612763,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-11-01,21300032,2013,1610612747,85.0,0.366,0.708,0.296,22.0,46.0,1610612759,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
2013-11-02,21300039,2013,1610612757,115.0,0.556,0.895,0.400,29.0,39.0,1610612759,...,0.0000,0.4516,0.0000,0.7332,0.0000,0.4084,0.0,23.0,0.0,44.4
